In [1]:
from selenium import webdriver
# from msedge.selenium_tools import Edge, EdgeOptions
import chromedriver_binary
import csv

In [2]:
#chrome
driver = webdriver.Chrome()
#Firefox
# driver = webdriver.Firefox()
#MSEdge
# options = EdgeOptions()
# options.use_chromium = True
# driver = Edge(options=options)

In [3]:
url = 'https://www.amazon.com/'
driver.get(url)

In [4]:
def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}'
    keyword = keyword.replace(' ', '+')
    return temp.format(keyword)

In [5]:
url = my_url('laptop charger')

In [6]:
url

'https://www.amazon.com/s?k=laptop+charger'

In [7]:
driver.get(url)

In [8]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
def change_page(page):
    temp = url 
    temp += '&page={}'.format(page)
    return temp

In [10]:
bs_result = bs.findAll('div', {'data-component-type': 's-search-result'})

In [11]:
bs_result[0].h2.a

<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&amp;adId=A09632352LYK70T6I3LXY&amp;qualifier=1672898402&amp;id=3934022438155897&amp;widgetName=sp_atf&amp;url=%2FCompatible-15-1039wm-15-1033wm-15-w117cl-15-w237cl%2Fdp%2FB08JHYX5XS%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dlaptop%2Bcharger%26qid%3D1672898402%26sr%3D8-1-spons%26psc%3D1%26smid%3DAIJ5GN3RZJLRO"><span class="a-size-medium a-color-base a-text-normal">65W Laptop Charger for HP Pavilion x360 11 13 15, Envy x360 13 15 17, 15-f111dx 15-f211wm 15-f233wm 15-f278nr 15-r052nr 15-r132wm Power Cord 19.5V 3.33A Charging Cord</span> </a>

In [12]:
def extract_records(obj):
    atag = obj.h2.a
    des = obj.h2.a.text.strip()
    url = 'https://www.amazon.com/' + atag.get('href')
    try:
        parent = obj.find('span', 'a-price')
        price = parent.find('span', 'a-offscreen').text
    except AttributeError:
        return 
    try:
        rate = obj.find('span', 'a-icon-alt').text
        count_review = obj.find('span', {'class': 'a-size-base', 'dir': 'auto'})
    except AttributeError:
        rate = ''
        count_review = ''
    image = obj.find('img', {'class': 's-image'}).get('src')
    
    return (des, price, rate, count_review, url, image)

extract_records(bs)

('65W\xa0Laptop Charger for HP\xa0Pavilion x360 11 13 15, Envy x360\xa013 15 17, 15-f111dx 15-f211wm 15-f233wm 15-f278nr 15-r052nr 15-r132wm Power Cord 19.5V 3.33A\xa0Charging Cord',
 '$12.57',
 '4.4 out of 5 stars',
 None,
 'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09632352LYK70T6I3LXY&qualifier=1672898402&id=3934022438155897&widgetName=sp_atf&url=%2FCompatible-15-1039wm-15-1033wm-15-w117cl-15-w237cl%2Fdp%2FB08JHYX5XS%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dlaptop%2Bcharger%26qid%3D1672898402%26sr%3D8-1-spons%26psc%3D1%26smid%3DAIJ5GN3RZJLRO',
 'https://m.media-amazon.com/images/W/WEBP_402378-T2/images/I/71PSZzZNd8L._AC_UY218_.jpg')

In [13]:
def main(keyword):
    driver = webdriver.Chrome()
    records = []
    url = my_url(keyword)
    
    for page in range(1, 3):
        driver.get(change_page(page))
        from bs4 import BeautifulSoup
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        bs_result = bs.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in bs_result:
            record = extract_records(item)
            if record:
                records.append(record)
                
        with open('result.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Description', 'Price', 'Rate', 'Review Counts', 'URL', 'Image'])
            writer.writerows(records)
        
main('laptop')